In [25]:
# RAG(검색 증강 생성) 시스템의 핵심 구성 요소인 문서 로드, 분할, 임베딩 모델 준비, 그리고 벡터 데이터베이스(ChromaDB) 초기화
# text data로 RAG 구현
# data(file) loading -> embedding -> vectorDB 저장 -> vectorDB에서 검색 -> prompt 내용 강화 -> LLM에 질문 -> 결과 얻기

!pip install -U langchain langchain-community chromadb langchain-google-genai google-genai langchain-openai python-dotenv
!pip install -U sentence-transformers
!pip install -U langchain-text-splitters


In [26]:
import os, io
from dotenv import load_dotenv # 환경 변수 파일(.env) 로드
from sentence_transformers import SentenceTransformer # 텍스트를 벡터로 변환하는 임베딩 모델 라이브러리
import chromadb # 벡터 데이터베이스 (Vector Database) 라이브러리
from chromadb.config import Settings # ChromaDB 설정 관련 (현재 코드에서는 사용되지 않음)
from google.colab import userdata # Colab의 보안 저장소에서 API 키를 가져오기 위함
import google.generativeai as genai # Google Gemini API를 사용하기 위한 SDK

load_dotenv()

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

llm = genai.GenerativeModel("gemini-2.5-flash") # 모델 이름을 직접 전달
embedder = SentenceTransformer("all-MiniLM-L6-v2")    # 'all-MiniLM-L6-v2': 빠르고 성능이 좋은 범용 임베딩 모델 (RAG 시스템에서 텍스트를 벡터로 변환)
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html



# --- 텍스트 로드 및 분할 방법 4가지 비교 ---


# # text 읽기1 - raw  (직접 리스트에 텍스트를 정의)
# documents = [
#     "김치찌개는 한국의 대표적인 찌개요리이다.",
#     "된장찌개는 발효된 된장을 이용해 만든다.",
#     "비빔밥은 여러 가지 나물을 비벼서 먹는 밥 요리이다.",
#     "불고기는 양념한 소고기를 구워 먹는 전통 음식이다."
#     "삼계탕은 닭에 인삼, 대추 생강 등을 넣고 푹 끓인 보양식이다."
# ]

# # text 읽기2 (.txt 파일)
# with open("foods.txt", "r", encoding="utf-8")as f:
#   documents = [line.strip() for line in f if line.strip()]

# # text 읽기3 (Langchain 방식 - 파일 전체를 하나의 Document로 로드))
# from langchain_community.document_loaders import TextLoader
# loader = TextLoader("foods.txt", encoding="utf-8")  # 파일 전체를 하나의 Document 객체로 로드 (Document에는 내용과 메타데이터가 포함)
# datas = loader.load()
# print(datas)
# # documents = [doc.page_content for doc in datas]     # Document 객체에서 순수 텍스트(page_content)만 추출

# # 2줄은 아래 text 읽기 4와 같아짐
# documents = datas[0].page_content.split("\n")
# documents = [doc.strip() for doc in documents if doc.strip()]

# text 읽기4 (Langchain 방식 - 줄마다 분리)
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter  # 텍스트를 특정 기준으로 분할하는 분할기
loader = TextLoader("foods.txt", encoding="utf-8")
datas = loader.load()
# 텍스트 분할기 설정: '\n' (줄 바꿈) 기준으로 분할하며, chunk_size와 chunk_overlap은 줄 단위로 적용
splitter = CharacterTextSplitter(separator='\n', chunk_size=100, chunk_overlap=0)
spl_docs = splitter.split_documents(datas)           # Document 객체를 분할
documents = [doc.page_content for doc in spl_docs]  # 분할된 Document 객체에서 텍스트 추출
text = datas[0].page_content
lines = text.split('\n')
max_len = max(len(line) for line in lines if line.strip())
splitter = CharacterTextSplitter(separator='\n', chunk_size=max_len, chunk_overlap=0)
chunks = splitter.split_text(text)
document = [c.strip() for c in chunks if c.strip()]

print(len(documents))
print(documents)

3
['documents = [\n    김치찌개는 한국의 대표적인 찌개요리이다.\n    된장찌개는 발효된 된장을 이용해 만든다.', '비빔밥은 여러 가지 나물을 비벼서 먹는 밥 요리이다.\n    불고기는 양념한 소고기를 구워 먹는 전통 음식이다.', '삼계탕은 닭에 인삼, 대추 생강 등을 넣고 푹 끓인 보양식이다\n]']


추가적으로 알아야 할 핵심 개념 (RAG의 기초)
이 코드는 RAG (Retrieval-Augmented Generation) 시스템의 데이터 준비 단계를 완벽하게 보여줌.

학습 목표인 텍스트를 위한 딥러닝 (단어 임베딩, 트랜스포머 아키텍처)과 직접적으로 연결되는 중요한 내용.

1. 텍스트 임베딩과 SentenceTransformer
    단어 임베딩 모델을 배운 후 그 개념을 문장/문서 레벨로 확장하여 활용.

    임베딩 (Embedding): 텍스트를 의미를 내포한 고차원 벡터 (High-Dimensional Vector)로 변환하는 과정. "김치찌개"와 "된장찌개"의 임베딩 벡터는 "불고기"의 임베딩 벡터보다 벡터 공간에서 더 가깝게 위치.

    SentenceTransformer: 문장 단위의 임베딩에 특화된 트랜스포머 기반 모델.

    사용 이유: 검색 시스템(RAG)에서는 단순히 단어의 의미뿐만 아니라 문장 전체의 의미(문맥)가 유사한 문서를 찾는 것이 중요합니다. SentenceTransformer는 이러한 문장 유사도(Semantic Similarity)를 계산하는 데 최적화.

    "all-MiniLM-L6-v2"는 BERT의 변형인 MiniLM을 기반으로 하며, 속도와 성능 균형이 뛰어나 RAG 초기 모델로 많이 사용.

2. 벡터 데이터베이스 (Vector Database) - ChromaDB

    역할: 임베딩 모델이 생성한 벡터들을 저장하고, 사용자의 질문 벡터와 가장 유사한 벡터(문서)를 빠르게 검색하는 데 특화된 데이터베이스.

    작동 원리: ChromaDB는 내부적으로 HNSW (Hierarchical Navigable Small World)와 같은 근접 이웃 탐색(Approximate Nearest Neighbors, ANN)알고리즘을 사용하여 수십억 개의 벡터 중에서도 의미적으로 가장 가까운 벡터들을 효율적으로 찾아냄.

    활용: 이 코드 다음에 해야 할 일은 documents 리스트의 각 문장을 embedder로 벡터화한 후, 이 벡터들과 원본 텍스트를 ChromaDB에 저장(인덱싱).

3. LangChain의 역할과 문서 처리 (Document Loading & Splitting)

    RAG에서 데이터 준비는 매우 중요. LangChain은 이 과정을 표준화하고 쉽게 만듦.

    Document 객체: LangChain에서 텍스트는 단순 문자열이 아니라 Document라는 객체로 다루어짐. 이 객체는 핵심 내용(page_content) 외에도 출처(source), 페이지 번호 등 메타데이터(Metadata)를 함께 포함하여, LLM이 응답의 근거(Source)를 제시할 수 있도록 도움.

    Text Loader: 파일 형식(TXT, PDF, HTML 등)에 관계없이 데이터를 읽어 Document 객체로 변환하는 역할.

    Text Splitter (텍스트 분할기): 사용자님의 예시처럼, 전체 문서를 작은 단위인 청크(Chunk)로 쪼갬.

    이유: LLM은 한 번에 처리할 수 있는 입력 길이(컨텍스트 윈도우)에 제한이 있음. 긴 문서를 통째로 넣는 대신, 질문과 관련된 작은 청크만 검색하여 LLM에게 제공해야 함.

    CharacterTextSplitter는 가장 기본적인 분할기로, 특정 구분자(\n)를 사용하여 텍스트를 나눔. 실제 대용량 RAG 시스템에서는 의미의 경계를 보존하는 RecursiveCharacterTextSplitter 등을 주로 사용.

    chunk_size와 chunk_overlap:

    chunk_size: 분할된 텍스트 조각의 최대 길이.
    chunk_overlap: 텍스트 조각들 사이에 겹치는 부분. 이는 문장의 문맥이 끊기는 것을 방지하여, LLM이 더 정확하게 답변할 수 있도록 도움. 현재 코드는 chunk_overlap=0으로 설정되어 있음.

결론적으로, 이 코드를 통해 RAG 시스템에서 비정형 텍스트 데이터를 검색 가능한 구조화된 벡터 데이터로 변환하는 단계

In [27]:
# 임베딩 벡터로 변환
doc_embeddings = embedder.encode(documents)
print(doc_embeddings[0][:5])

# ChromaDB에 저장
# 방법1 : 임시저장
# chroma_clinent = chromadb.Client(Settings(anonymized_telemetry=False))

# 방법2 : 영구저장
chroma_client = chromadb.Client(Settings(
    persist_directory = "./chroma_db",   # DB 파일을 `./chroma_db` 폴더에 저장하여 영구 보존 ( ./. hidden)
    anonymized_telemetry=False  # 보안 강화 목적

))

collection = chroma_client.get_or_create_collection(name="foods")

for i, (doc, embedding) in enumerate(zip(documents, doc_embeddings)):
    collection.add(
        documents = [doc],
        embeddings = [embedding.tolist()],  # NumPy 배열을 리스트로 변환하여 저장
        ids = [f"doc_{i}"]
    )

[-0.03326124  0.16256729  0.05103043  0.0360138   0.02336872]


RAG에서의 역할: 이 단계는 외부 지식 저장소(Knowledge Base)를 구축하는 것.

embedder.encode(documents): Sentence-Transformers 모델(예: MiniLM)을 사용하여 문서를 벡터화.

persist_directory: DB를 영구 저장하여, 코드를 다시 실행해도 이전에 저장한 'foods' 컬렉션의 데이터가 유지되도록.

collection.add(): 문서 내용(doc)과 그에 해당하는 벡터(embedding)를 ids와 함께 벡터 DB에 색인(Indexing).

In [28]:
# 여기서부터 RAG 단계를 따라 처리 ----------
# RAG 흐름 1단계 : Retrival- 관련 문서 검색
query = " 한국의 대표적인 찌개 음식을 알려줘"
query_embedding = embedder.encode(query) # [0]  # 질문을 벡터로 변환 (문서와 같은 공간에 위치)

results = collection.query(
    query_embeddings = [query_embedding.tolist()],  # 질문 벡터로 검색
    # n_results = 3,
    n_results=len(documents), # 저장된 모든 문서 수만큼 반환 요청 (가장 유사한 순서대로)
    include=["documents", "distances"]
)
print("results :", results)

# 유사도 거리 확인
import numpy as np
cosine_distances = results["distances"][0]
print("코사인 거리 :" , cosine_distances)

# 유사도 직접 계산
similarities = []
for doc_id in results["ids"][0]:
  doc_embed = collection.get(ids=[doc_id], include=["embeddings"])["embeddings"][0]
  doc_embed = np.array(doc_embed, dtype=float)
  # np.doc 사용 여부 확인 : 정규화 확인 _ 1.0에 근사하면 OK
  # 정규화 확인: 임베딩 모델은 보통 벡터의 크기(L2 Norm)를 1.0으로 정규화(Normalization)
  print("정규화 확인 1 : ", np.linalg.norm(query_embedding))
  print("정규화 확인 2 : ", np.linalg.norm(doc_embed))

  sim = np.dot(query_embedding, doc_embed)
  similarities.append(sim)

print("유사도 값 확인 : " , similarities)

results : {'ids': [['doc_1', 'doc_2', 'doc_0']], 'embeddings': None, 'documents': [['비빔밥은 여러 가지 나물을 비벼서 먹는 밥 요리이다.\n    불고기는 양념한 소고기를 구워 먹는 전통 음식이다.', '삼계탕은 닭에 인삼, 대추 생강 등을 넣고 푹 끓인 보양식이다\n]', 'documents = [\n    김치찌개는 한국의 대표적인 찌개요리이다.\n    된장찌개는 발효된 된장을 이용해 만든다.']], 'uris': None, 'included': ['documents', 'distances'], 'data': None, 'metadatas': None, 'distances': [[0.5289239883422852, 0.5785448551177979, 1.1728326082229614]]}
코사인 거리 : [0.5289239883422852, 0.5785448551177979, 1.1728326082229614]
정규화 확인 1 :  0.99999994
정규화 확인 2 :  1.0000000057480987
정규화 확인 1 :  0.99999994
정규화 확인 2 :  0.9999999269358497
정규화 확인 1 :  0.99999994
정규화 확인 2 :  1.0000000963105489
유사도 값 확인 :  [np.float64(0.7355379676762908), np.float64(0.7107274634863904), np.float64(0.4135837698869711)]


코사인 거리 vs. 유사도:

거리(Distance): ChromaDB가 반환한 값. 작을수록 유사함.

유사도(Similarity): 우리가 직접 계산한 값(내적). 1에 가까울수록 유사함.

핵심: 코사인 거리와 1 - 코사인 유사도는 강한 연관. 이 과정을 통해 검색이 의도대로 (질문과 관련된 문서가 높은 유사도 순으로) 이루어졌는지 검증.

# 코사인 유사도($\text{sim}$)는 정규화된 벡터의 내적($\vec{A} \cdot \vec{B}$)과 같습니다.

# $\text{sim} = \frac{\vec{A} \cdot \vec{B}}{||\vec{A}|| \cdot ||\vec{B}||}$


# $||\vec{A}|| \approx 1, ||\vec{B}|| \approx 1$ 이므로, $\text{sim} \approx \vec{A} \cdot \vec{B}$


In [32]:
# RAG 흐름 , 2,3단계 : 증강(Augmented) 및 생성(Generation)
# 똑똑한 프롬프트 만들기
# 검색된 문자열 하나로 합치기
retrieved_docs_list = results["documents"][0]
retrieved_docs = "\n\n".join(retrieved_docs_list)
# print(retrieved_docs)

prompt = f"""
너는 한국 전통 음식에 대해 잘 아는 전문가야.
지금부터 사용자 질문에 답변할 때는 반드시 아래 내용을 참고해서 답변해 줘
{retrieved_docs}

위 내용을 참고하여 '{query}'에 대해 대답해 줘.
10문장 이내로, 마크다운(**,*,_ 등) 같은 스타일 없이 평문으로 답해 줘
"""

# LLM 호출을 추가하여 'response' 변수를 정의.
response = llm.generate_content(prompt)

print('LLM 응답 :')
# print(response.txt)
if hasattr(response, "text"):
  text = response.text
else:
  text = response.candidates[0].content.parts[0].text

# 문장 분리
import re
sentences = re.split(r'\.\s+', text)
for s in sentences:
  s = s.strip()
  if not s:
    continue
  if not s.endswith("."):
    s += "."

    print(s)

LLM 응답 :
한국의 대표적인 찌개 음식에 대해 설명해 드리겠습니다.
한국의 찌개 요리 중 대표적인 것은 김치찌개입니다.
김치찌개는 한국의 대표적인 찌개요리입니다.
또한 된장찌개도 한국의 매우 중요한 찌개 중 하나입니다.
된장찌개는 발효된 된장을 이용해 만들어지는 것이 특징입니다.


증강(Augmentation): 검색된 문서(retrieved_docs)를 프롬프트 안에 삽입하여 LLM의 기존 지식 외에 외부의 최신/특정 지식을 제공하는 과정.

생성(Generation): LLM이 제공된 증강 정보를 기반으로 사용자의 질문에 대한 최종 답변을 생성하는 단계.


프롬프트 역할: LLM에게 역할(전문가)을 부여하고, 참조할 자료({retrieved_docs})를 명확히 제시하며, 제약 조건(10문장 이내)을 걸어 출력 품질을 제어하는 프롬프트 엔지니어링 기법이 적용.

- RAG 시스템 심화 학습 내용

1. 청킹 전략의 중요성RAG 성능 향상을 위해 긴 문서를 적절히 나누는 청킹 작업이 필수적임. 청크가 너무 길면 불필요한 정보가 섞여 검색 유사도 저하를 유발함. 너무 짧으면 LLM 답변을 위한 문맥(Context)이 부족함. 해결책으로 문맥 끊김 방지를 위한 슬라이딩 윈도우 기법 또는 의미적 경계 기준 분할인 시맨틱 청킹을 적용해야 함. 목표는 청크 하나가 독립된 하나의 의미를 갖도록 하는 것임.

2. 임베딩 모델 선택 및 Fine-Tuning범용 모델(예: MiniLM-L6-v2)은 특정 도메인(예: 내부 자료)의 용어와 문맥 이해도가 낮아 검색 정확도 하락 가능성 있음. 따라서 한국어 성능 검증 모델 (예: KC-BERT)을 선택하거나, 전이 학습을 활용하여 내부 데이터로 임베딩 모델을 Fine-Tuning해야 함. 이 과정이 해당 도메인의 검색 정확도(유사도)를 획기적으로 높이는 핵심 방법임.

3. RAG 검색 최적화 (Advanced Retrieval)단순히 상위 $K$개 문서만 가져오는 것을 넘어, 검색 품질 향상 필요함. 이를 위해 HyDE (Hypothetical Document Embedding) 기법을 사용함. 이는 질문 대신 LLM이 생성한 '가상의 답변'을 임베딩하여 벡터 DB를 검색하는 방식임. 또한, 검색된 후보 문서를 더 정교한 언어 모델로 재순위화(Re-ranking)하는 과정을 거쳐 최종적으로 가장 정확한 문서를 선택함.

4. 시스템 통합 (Django/Flask 연동)구축된 RAG 시스템의 최종 목표는 웹 서비스 배포임. 실무에서는 ChromaDB 클라이언트를 웹 서버 환경에 통합해야 함. Django 또는 Flask 서버 시작 시 PersistentClient를 통해 DB에 연결하고, 사용자 질문이 들어오면 웹 서버 코드가 collection.query()를 실행하여 검색 결과를 LLM API로 전달하는 방식으로 RAG 파이프라인을 구현하고 서비스화해야 함.